In [29]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
import numpy as np
from netgen.csg import unit_cube
from ngsolve import Mesh,VOL,specialcf

ngmesh = Mesh(unit_cube.GenerateMesh(maxh=0.25))

m=6

mesh = UnstructuredHypertriangleMesh(m,ngmesh)
print(len(mesh.points))
print(len(mesh.vertices))
print(mesh.hypercells)
print(len(mesh.hypercells))

987
987
[[101 104 137 135 276]
 [242 245 278 276 417]
 [383 386 419 417 558]
 ...
 [447 588 611 612 681]
 [588 729 752 753 822]
 [729 870 893 894 963]]
10920


In [30]:
print(len(mesh.points))
print(len(mesh.vertices))

987
987


In [31]:
from methodsnm.fes import *
fes = P1_Hypertriangle_Space(mesh)

print(len(mesh.vertices))
print(len(mesh.bndry_vertices))

987
807


In [32]:
from methodsnm.forms import *
from methodsnm.formint import *
from numpy import pi, cos ,sin

blf = BilinearForm(fes)
c = GlobalFunction(lambda x: 1, mesh = mesh)
blf += LaplaceIntegral(c)
blf.assemble()

lf = LinearForm(fes)
f = GlobalFunction(lambda x: 4*pi**2*cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), mesh = mesh)
lf += SourceIntegral(f)
lf.assemble()

In [33]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

In [34]:

uh = FEFunction(fes)
res = lf.vector - blf.matrix.dot(uh.vector)
from scipy.sparse.linalg import spsolve
from methodsnm.solver import solve_on_freedofs
freedofs = list_diff(mesh.vertices,mesh.bndry_vertices)
uh._set(lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), True)
#uh.vector = spsolve(blf.matrix, lf.vector)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)


In [35]:
from methodsnm.forms import compute_difference_L2

uex =  GlobalFunction(lambda x: cos(pi*x[0])*cos(pi*x[1])*cos(pi*x[2])*cos(pi*x[3]), mesh = mesh)
l2diff = compute_difference_L2(uh, uex, mesh, intorder = 3)

print("l2diff =", l2diff)

l2diff = 0.07075343211420215
